In [1]:
import sqlite3
import pandas as pd

In [2]:
providers_df = pd.read_csv('providers_data.csv')
receivers_df = pd.read_csv('receivers_data.csv')
food_df = pd.read_csv('food_listings_data.csv')
claims_df = pd.read_csv('claims_data.csv')

In [3]:
conn = sqlite3.connect('food_storage.db')

In [4]:
providers_df.to_sql('Providers', conn, if_exists='replace', index=False)
receivers_df.to_sql('Receivers', conn, if_exists='replace', index=False)
food_df.to_sql('Food_Listings', conn, if_exists='replace', index=False)
claims_df.to_sql('Claims', conn, if_exists='replace', index=False)

1000

In [5]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

            name
0      Providers
1      Receivers
2  Food_Listings
3         Claims


In [6]:
pd.read_sql_query("SELECT * FROM Providers LIMIT 5", conn)

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


In [7]:
pd.read_sql_query("SELECT * FROM Receivers LIMIT 5", conn)

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154


In [8]:
pd.read_sql_query("SELECT * FROM Food_Listings LIMIT 5", conn)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner


In [9]:
pd.read_sql_query("SELECT * FROM Claims LIMIT 5", conn)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17


In [10]:
# Convert 'Expiry_Date' and 'Timestamp' columns to proper datetime
food_df['Expiry_Date'] = pd.to_datetime(food_df['Expiry_Date'], errors='coerce')
claims_df['Timestamp'] = pd.to_datetime(claims_df['Timestamp'], errors='coerce')

In [11]:
food_df.to_sql('Food_Listings', conn, if_exists='replace', index=False)
claims_df.to_sql('Claims', conn, if_exists='replace', index=False)

1000

In [12]:
pd.read_sql_query("SELECT * FROM Food_Listings LIMIT 5", conn)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17 00:00:00,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24 00:00:00,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,2025-03-28 00:00:00,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,2025-03-16 00:00:00,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,2025-03-19 00:00:00,279,Restaurant,Garciaport,Vegan,Dinner


In [13]:
pd.read_sql_query("SELECT * FROM Claims LIMIT 5", conn)

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00
2,3,626,492,Completed,2025-03-21 00:59:00
3,4,61,933,Cancelled,2025-03-04 09:08:00
4,5,345,229,Pending,2025-03-14 15:17:00


### SQL Queries & Analysis

1. How many food providers and receivers are there in each city?

In [14]:
query = '''
SELECT 
    City,
    COUNT(CASE WHEN source_table = 'Provider' THEN 1 END) AS Provider_Count,
    COUNT(CASE WHEN source_table = 'Receiver' THEN 1 END) AS Receiver_Count,
    COUNT(*) AS Total_Count
FROM (
    SELECT City, 'Provider' AS source_table FROM Providers
    UNION ALL
    SELECT City, 'Receiver' AS source_table FROM Receivers
) AS combined_data
GROUP BY City
ORDER BY City;
'''
pd.read_sql_query(query, conn)

,City,Provider_Count,Receiver_Count,Total_Count
0,Aaronshire,0,1,1
1,Adambury,1,0,1
2,Adamland,0,1,1
3,Adamsview,1,0,1
4,Adamsville,1,0,1
...,...,...,...,...
1873,Yatesside,1,0,1
1874,Youngchester,1,0,1
1875,Zacharyview,0,1,1
1876,Zimmermanton,1,0,1


2. Which type of food provider contributes the most food?

In [15]:
query = '''
SELECT Provider_Type, COUNT(*) AS Total_Contributions
FROM Food_Listings
GROUP BY Provider_Type
ORDER BY Total_Contributions DESC;
'''
pd.read_sql_query(query, conn)

,Provider_Type,Total_Contributions
0,Supermarket,267
1,Restaurant,258
2,Grocery Store,243
3,Catering Service,232


3. What is the contact information of food providers in a specific city?

In [16]:
query = '''
SELECT Name, Contact
FROM Providers
WHERE City = 'Zimmermanville';
'''
pd.read_sql_query(query, conn)

,Name,Contact
0,"Barnes, Castro and Curtis",507.577.7566


4. Which receivers have claimed the most food?

In [17]:
query = '''
SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
FROM Claims c
JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Receiver_ID
ORDER BY Total_Claims DESC
LIMIT 20;
'''
pd.read_sql_query(query, conn)

,Name,Total_Claims
0,Anthony Garcia,5
1,Matthew Webb,5
2,William Frederick,5
3,Scott Hunter,5
4,Donald Caldwell,4
5,Alexandra Owens,4
6,Chelsea Powell,4
7,Betty Reid,4
8,Kristine Martin,4
9,Kristina Simpson,4


5. What is the total quantity of food available from all providers?

In [18]:
query = '''
SELECT SUM(Quantity) AS Total_Quantity_Available
FROM Food_Listings;
'''
pd.read_sql_query(query, conn)

,Total_Quantity_Available
0,25794


6. Which city has the highest number of food listings?

In [19]:
query = '''
SELECT Location AS City, COUNT(*) AS Number_of_Listings
FROM Food_Listings
GROUP BY Location
ORDER BY Number_of_Listings DESC
LIMIT 1;
'''
pd.read_sql_query(query, conn)

,City,Number_of_Listings
0,South Kathryn,6


7. What are the most commonly available food types?

In [20]:
query = '''
SELECT Food_Type, COUNT(*) AS Count
FROM Food_Listings
GROUP BY Food_Type
ORDER BY Count DESC;
'''
pd.read_sql_query(query, conn)

,Food_Type,Count
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


8. How many food claims have been made for each food item?

In [21]:
query = '''
SELECT 
    f.Food_Name,
    f.Food_ID,
    COUNT(c.Claim_ID) AS Total_Claims
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
GROUP BY c.Food_ID
ORDER BY Total_Claims DESC;
'''
pd.read_sql_query(query, conn)

,Food_Name,Food_ID,Total_Claims
0,Fish,548,5
1,Chicken,486,5
2,Soup,463,5
3,Dairy,981,4
4,Salad,925,4
...,...,...,...
642,Fruits,13,1
643,Pasta,11,1
644,Dairy,7,1
645,Vegetables,6,1


9. Which provider has had the highest number of successful food claims?

In [22]:
query = '''
SELECT 
    p.Name AS Provider_Name,
    COUNT(c.Claim_ID) AS Completed_Claims
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
JOIN Providers p ON f.Provider_ID = p.Provider_ID
WHERE c.Status = 'Completed'
GROUP BY p.Provider_ID
ORDER BY Completed_Claims DESC
LIMIT 1;
'''
pd.read_sql_query(query, conn)

,Provider_Name,Completed_Claims
0,Barry Group,5


10. What percentage of food claims are Completed vs Pending vs Cancelled?

In [23]:
query = '''
SELECT 
    Status,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Claims) AS Percentage
FROM Claims
GROUP BY Status;
'''
pd.read_sql_query(query, conn)

,Status,Percentage
0,Cancelled,33.6
1,Completed,33.9
2,Pending,32.5


11. What is the average quantity of food claimed per receiver?

In [24]:
query = '''
SELECT 
    r.Name AS Receiver_Name,
    AVG(f.Quantity) AS Avg_Quantity_Claimed
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY c.Receiver_ID
ORDER BY Avg_Quantity_Claimed DESC;
'''
pd.read_sql_query(query, conn)

,Receiver_Name,Avg_Quantity_Claimed
0,Thomas Villanueva,50.0
1,Nancy Jones,50.0
2,Christopher Wright,50.0
3,Peggy Knight,50.0
4,Daniel Williams,50.0
...,...,...
619,Corey Rodriguez,2.0
620,Matthew Sanchez,2.0
621,Matthew Johnson,2.0
622,Amanda Kline,1.0


12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

In [25]:
query = '''
SELECT 
    f.Meal_Type,
    COUNT(c.Claim_ID) AS Total_Claims
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Meal_Type
ORDER BY Total_Claims DESC;
'''
pd.read_sql_query(query, conn)

,Meal_Type,Total_Claims
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


13. What is the total quantity of food donated by each provider?

In [26]:
query = '''
SELECT 
    p.Name AS Provider_Name,
    SUM(f.Quantity) AS Total_Quantity_Donated
FROM Food_Listings f
JOIN Providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Provider_ID
ORDER BY Total_Quantity_Donated DESC;
'''
pd.read_sql_query(query, conn)

,Provider_Name,Total_Quantity_Donated
0,Barry Group,179
1,"Evans, Wright and Mitchell",158
2,Smith Group,150
3,Nelson LLC,142
4,Ruiz-Oneal,140
...,...,...
632,Mcgee PLC,1
633,Brown and Sons,1
634,"Martinez, Armstrong and Carroll",1
635,Gonzales-Moore,1


14. Which top 5 cities have the most completed food claims?

In [27]:
query = '''
SELECT 
    f.Location AS City,
    COUNT(c.Claim_ID) AS Completed_Claims
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed'
GROUP BY f.Location
ORDER BY Completed_Claims DESC
LIMIT 5;
'''
pd.read_sql_query(query, conn)

,City,Completed_Claims
0,South Kathryn,5
1,Zimmermanville,4
2,New Carol,4
3,East Heatherport,4
4,Devinmouth,4


15. Which food items have expired and were not claimed in time?

In [28]:
query = '''
SELECT *
FROM Food_Listings
WHERE DATE(Expiry_Date) < DATE('now');
'''
pd.read_sql_query(query, conn)

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17 00:00:00,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24 00:00:00,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,2025-03-28 00:00:00,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,2025-03-16 00:00:00,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,2025-03-19 00:00:00,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,2025-03-30 00:00:00,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,2025-03-18 00:00:00,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,2025-03-22 00:00:00,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,2025-03-30 00:00:00,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


16. Which food items have never been claimed by any receiver?

In [29]:
query = '''
SELECT f.Food_ID, f.Food_Name
FROM Food_Listings f
LEFT JOIN Claims c ON f.Food_ID = c.Food_ID
WHERE c.Claim_ID IS NULL;
'''
pd.read_sql_query(query, conn)

,Food_ID,Food_Name
0,2,Soup
1,4,Fruits
2,5,Soup
3,8,Rice
4,10,Rice
...,...,...
348,987,Dairy
349,993,Soup
350,994,Salad
351,996,Fish


17. Which cities have the highest number of food providers?

In [30]:
query = '''
SELECT City, COUNT(*) AS Provider_Count
FROM Providers
GROUP BY City
ORDER BY Provider_Count DESC;
'''
pd.read_sql_query(query, conn)

,City,Provider_Count
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


18. What is the average number of claims made per food item?

In [31]:
query = '''
SELECT 
    ROUND(AVG(Claim_Count), 2) AS Avg_Claims_Per_Item
FROM (
    SELECT Food_ID, COUNT(*) AS Claim_Count
    FROM Claims
    GROUP BY Food_ID
);
'''
pd.read_sql_query(query, conn)

,Avg_Claims_Per_Item
0,1.55


19. Which receivers have canceled the most food claims?

In [32]:
query = '''
SELECT 
    r.Name,
    COUNT(*) AS Cancelled_Claims
FROM Claims c
JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE c.Status = 'Cancelled'
GROUP BY c.Receiver_ID
ORDER BY Cancelled_Claims DESC
LIMIT 5;
'''
pd.read_sql_query(query, conn)

,Name,Cancelled_Claims
0,William Frederick,4
1,Scott Hunter,4
2,Michael Townsend,3
3,Scott Schroeder,3
4,Eric Mays,3


20. Which providers have donated more than 100 units of food?

In [33]:
query = '''
SELECT 
    p.Name, 
    SUM(f.Quantity) AS Total_Donated
FROM Providers p
JOIN Food_Listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Provider_ID
HAVING SUM(f.Quantity) > 100
ORDER BY Total_Donated DESC;
'''
pd.read_sql_query(query, conn)

,Name,Total_Donated
0,Barry Group,179
1,"Evans, Wright and Mitchell",158
2,Smith Group,150
3,Nelson LLC,142
4,Ruiz-Oneal,140
5,Blankenship-Lewis,124
6,Campbell LLC,123
7,Kelly-Ware,123
8,Bradford-Martinez,121
9,Shepherd and Sons,116


21. What is the average time gap between food claim and its expiry date?

In [34]:
query = '''
SELECT 
    ROUND(AVG(JULIANDAY(f.Expiry_Date) - JULIANDAY(c.Timestamp)), 2) AS Avg_Days_Before_Expiry
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
WHERE c.Status = 'Completed';
'''
pd.read_sql_query(query, conn)

,Avg_Days_Before_Expiry
0,11.37


22. Which food items have been claimed multiple times by different receivers?

In [35]:
query = '''
SELECT 
    f.Food_Name,
    COUNT(c.Claim_ID) AS Number_of_Claims
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
GROUP BY f.Food_ID
HAVING COUNT(c.Claim_ID) > 1
ORDER BY Number_of_Claims DESC;
'''
pd.read_sql_query(query, conn)

,Food_Name,Number_of_Claims
0,Fish,5
1,Chicken,5
2,Soup,5
3,Dairy,4
4,Salad,4
...,...,...
249,Rice,2
250,Salad,2
251,Chicken,2
252,Bread,2


23. What types of meals are most commonly claimed from a specific provider?

In [36]:
query = '''
SELECT 
    p.Name AS Provider_Name,
    f.Meal_Type,
    COUNT(c.Claim_ID) AS Total_Claims
FROM Claims c
JOIN Food_Listings f ON c.Food_ID = f.Food_ID
JOIN Providers p ON f.Provider_ID = p.Provider_ID
WHERE p.Name = 'Gonzales-Cochran'
GROUP BY f.Meal_Type
ORDER BY Total_Claims DESC;
'''
pd.read_sql_query(query, conn)

,Provider_Name,Meal_Type,Total_Claims
0,Gonzales-Cochran,Dinner,2
1,Gonzales-Cochran,Breakfast,1
